<a name="top" id="top"></a>

<div align="center">
    <h1>Quantum Drunkard's Walk</h1>
    <a href="https://github.com/AlbertLee0125">Albert Lee (Joon Lee)</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <br>
    <a href="https://github.com/pedromxavier">Pedro Maciel Xavier</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Computer Science &amp; Systems Engineering Program, Federal University of Rio de Janeiro</i>
    <br>
    <i>PSR Energy Consulting &amp; Analytics</i>
    <br>
    <br>
    <a href="https://colab.research.google.com/github/SECQUOIA/QC-Tutorial/blob/main/notebooks/quantum_walk.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
    <a href="#installation">
        <img src="https://img.shields.io/badge/⚙️-Installation_Instructions-teal" alt="Installation Instructions"/>
    </a>
    <a href="https://bernalde.github.io/">
        <img src="https://img.shields.io/badge/🌲-SECQUOIA-gold" alt="SECQUOIA"/>
    </a>
</div>



In [1]:
import Pkg

Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `C:\Users\pedroxavier\Documents\QC-Tutorial\notebooks`


In [2]:
using Conda

Conda.pip_interop(true)
Conda.pip("install", "webio_jupyter_extension")

[ Info: Running `conda config --set pip_interop_enabled true --file 'C:\Users\pedroxavier\.julia\conda\3\condarc-julia.yml'` in root environment
[ Info: Running `pip install webio_jupyter_extension` in root environment


In [3]:
using QCTutorial
using Plots
using Interact

# Make plots look professional
Plots.default(;
    fontfamily = "Computer Modern",
    plot_titlefontsize  = 16,
    titlefontsize       = 14,
    guidefontsize       = 12,
    legendfontsize      = 10,
    tickfontsize        = 10,
)

WebIO._IJuliaInit()

In [4]:
x = sum(Ket.(0:3, 2)) * (1 / 2)

0.5 |00⟩ + 0.5 |01⟩ + 0.5 |10⟩ + 0.5 |11⟩

In [5]:
y = sum(Bra.(0:3, 2)) * (1 / 2)

0.5 ⟨00| + 0.5 ⟨01| + 0.5 ⟨10| + 0.5 ⟨11|

In [6]:
Ket(0, 2)

 |00⟩

In [7]:
H(2) * Ket(0, 2)

0.5 |00⟩ + 0.5 |01⟩ + 0.5 |10⟩ + 0.5 |11⟩

## Random Walks

Random walks are stochastic processes over a graph $G$.

# Drunkard's Walk

The drunkard's walk is a random walk on a line.
At each step, the drunkard moves left or right with equal probability.
The drunkard's walk is a Markov chain, where the state is the position of the drunkard.

In [10]:
function random_walk(n::Integer=100; steps::Integer=25, num_reads::Integer=100n)
    # @assert steps < n ÷ 2 "`steps` must be less than `n ÷ 2`"

    # line positions
    m = n ÷ 2
    x = zeros(Int, n)

    # initial positions
    s = fill(m, num_reads)

    # random steps for every read
    # 2 * r - 1 takes from {0, 1} to {-1, 1}
    # sum(r) accounts the total movement for each read.
    # s + r broadcasts the sum, giving the final position
    r = sum(rand((-1, +1), steps, num_reads); dims=1)
    K = r + s'

    # x[s + r] += 1 adds 1 for every final position on the line
    for k in K
        (0 < k <= n) && (x[k] += 1)
    end

    # normalizes probability
    return x / num_reads
end

random_walk (generic function with 2 methods)

## Quantum Drunkard

In [11]:
function quantum_walk(n::Integer=100; steps::Integer=25, num_reads::Integer=100n, coin = (1/√2) * (ket"0" + ket"1"))
    m = n ÷ 2
    
    # Initial Coin
    σ = coin
    
    # Number of Qubits
    nq = ceil(Int, log2(n))
    
    # Initial Position
    x = Ket(m, nq)
    
    # Initial System State
    ψ = σ ⊗ x
    
    # Coin Operator
    C = H(1) ⊗ I(nq)
    
    # Shift Operator
    S = shift(n)
    
    # Coin-Shift Operator
    U = S * C
    
    # Apply U `steps` times
    for _ = 1:steps
        ψ = U * ψ
    end
    
    # Amplitues
    λ = zeros(Float64, n)

    for k = 1:n
        ϕ = Ket(k, nq)
        M = I(1) ⊗ (ϕ * ϕ')
        Ψ = M * ψ
        
        λ[k] = real(Ψ' * Ψ)
    end
    
    return λ
end

quantum_walk (generic function with 2 methods)

In [12]:
total_steps = 100

@manipulate for steps in 1:100,
        ymax = 0.2:0.1:1.0,
        show_random_walk in checkbox("Random Walk", value = true),
        show_quantum_walk in checkbox("Quantum Walk", value = true),
        show_quantum_walk_mirr in checkbox("Quantum Walk (Mirrored)", value = true),
        show_quantum_walk_symm in checkbox("Quantum Walk (Symmetric)", value = true)
    
    plt = plot(;
        plot_title="Random Walk vs. Quantum Walk",
        label="steps = $steps",
        ylims=(0.0,ymax),
        ylabel="Probability",
        xlims=(1,total_steps),
#         legend=:outerright,
    )
    
    if show_random_walk
        rw = random_walk(total_steps; steps)

        plot!(plt, rw; label="Random Walk", color=:violet)
    end
    
    if show_quantum_walk
        qw = quantum_walk(total_steps; steps)

        plot!(plt, qw; label="Quantum Walk \$ \\sigma = \\frac{1}{\\sqrt{2}} \\left( |0\\rangle{} + |1\\rangle{} \\right) \$", color=:green)
    end
    
    if show_quantum_walk_mirr
        qwm = quantum_walk(total_steps; steps, coin = (1/√2) * (ket"0" - ket"1"))
            
        plot!(plt, qwm; label="Quantum Walk \$ \\sigma = \\frac{1}{\\sqrt{2}} \\left( |0\\rangle{} - |1\\rangle{} \\right) \$", color=:blue)
    end
    
    if show_quantum_walk_symm
        qws = quantum_walk(total_steps; steps, coin = (1/√2) * (ket"0" + im * ket"1"))
            
        plot!(plt, qws; label="Quantum Walk \$ \\sigma = \\frac{1}{\\sqrt{2}} \\left( |0\\rangle{} + i |1\\rangle{} \\right) \$", color=:red)
    end
    
    plt
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["steps"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 100, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(50), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\pedroxavier\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\pedroxavier\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\pedroxavier\\.julia\\packages\\InteractBase\\LJXv3\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\pedroxavier\\.julia\\packages\\InteractBase\\LJXv3\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\pedroxavier\\.julia\\packages\\Interact\\PENUy\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000001d6720b3e10, Task (runnable) @0x000001d6720b3e10), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 2019811314848)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"

<a name="installation" id="installation"></a>

# Installation

## Colab Instructions

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia and IJulia. This can take a couple of minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.
4. After reload, be sure to select the `Colab Julia` kernel by clicking on _Runtime_ > _Change runtime type_ > _Select `Colab Julia` in dropdown_ > _Save_.

If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 to 4.

In [ ]:
%%shell

bash <(curl -s "https://raw.githubusercontent.com/SECQUOIA/QC-Tutorial/main/scripts/install-colab-julia.sh")